In [1]:
import numpy as np
import cvxpy as cp
from cvxpygen import cpg
import math

In [2]:
delta = 0.05

# create random Bernoulli dataset
arm = np.array([3, 4]) / 5
theta_star = np.array([6, 8])
S = np.linalg.norm(theta_star, 2) + 1
Lt = lambda t: (1 + S/2) * (t - 1)

n = 4000
X = np.random.randn(n, 2)
mu = lambda x: 1 / (1 + np.exp(-x))
r = np.random.binomial(1, mu(X @ theta_star))


likelihood = lambda theta: cp.sum(cp.multiply(r, cp.logistic(-X @ theta)) + cp.multiply((1 - r), cp.logistic(X @ theta)))

# Obtain \hat{\theta} using norm-constrained logistic regression
theta = cp.Variable(2)

obj = likelihood(theta)
constraints = [cp.norm(theta, 2) <= S]
objective = cp.Minimize(obj)
problem = cp.Problem(objective, constraints)
problem.solve()
theta_hat = theta.value


# Solve UCB
theta2 = cp.Variable(2)

obj = cp.sum(cp.multiply(theta2, arm))
constraints = [cp.norm(theta2, 2) <= S, 
               likelihood(theta2) - likelihood(theta_hat) <= cp.log(1/delta) + 2*cp.log(max(math.e, math.e*S*Lt(n)))]

objective = cp.Maximize(obj)

problem = cp.Problem(objective, constraints)
problem.solve(verbose=True)
print(problem.status)
print(theta2.value)

                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Aug 03 01:37:31 AM: Your problem has 2 variables, 2 constraints, and 0 parameters.
(CVXPY) Aug 03 01:37:31 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 03 01:37:31 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 03 01:37:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 03 01:37:31 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 03 01:37:31 AM: Compiling problem (target solver=CLARABEL).
(